In [ ]:
import random
from deap import base, creator, tools, algorithms
import numpy as np

# Coordenadas de las ciudades
cities = [(0, 0), (1, 3), (4, 3), (6, 1), (3, 0)]
n_cities = len(cities)

# Calcular la distancia euclidiana entre dos ciudades
def distance(city1, city2):
    return np.sqrt((city1[0] - city2[0])**2 + (city1[1] - city2[1])**2)

# Crear la estructura del problema
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimización
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("indices", random.sample, range(n_cities), n_cities)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Función de evaluación
def evaluate(individual):
    dist = sum(distance(cities[individual[i]], cities[individual[i + 1]]) for i in range(n_cities - 1))
    dist += distance(cities[individual[-1]], cities[individual[0]])  # Volver al inicio
    return dist,

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxOrdered)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# Algoritmo genético
def main():
    random.seed(42)
    pop = toolbox.population(n=100)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", min)

    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.2, ngen=100, stats=stats, halloffame=hof, verbose=True)
    return pop, log, hof

if __name__ == "__main__":
    population, logbook, hof = main()
    print("Mejor solución:", hof[0])
    print("Distancia mínima:", evaluate(hof[0])[0])
